# 태스크 4: RetrieveAndGenerate API를 사용하여 Amazon Bedrock 지식 기반에서 가드레일 기능 테스트

이 태스크에서는 RetrieveAndGenerate API를 사용하여 Amazon Bedrock 지식 기반에서 가드레일 기능을 테스트합니다. AnyCompany Financial 10K 문서에 기반한 이전 노트북에서 사용된 기존 지식 기반을 활용합니다.

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **주의:** **Run** 메뉴에서 **Run All Cells** 옵션을 사용하는 것보다 각 코드 셀을 개별적으로 실행하는 것이 좋습니다. 모든 셀을 한꺼번에 같이 실행하면 커널 충돌 또는 재시작 같은 예기치 않은 동작이 발생할 수 있습니다. 셀을 1개씩 실행하면 실행 흐름을 더욱 잘 제어하고, 잠재적인 오류를 조기에 발견하며, 의도한 대로 코드가 실행되도록 할 수 있습니다.

## 태스크 4.1: 환경 설정

이 태스크에서는 boto3 클라이언트를 초기화하여 이 노트북의 환경을 설정합니다. 노트북 전체에서 **RetrieveAndGenerate** API를 활용하여 지식 기반 기능을 테스트할 수 있습니다.

1. 다음의 코드 셀을 실행해 boto3 클라이언트를 초기화하여 환경을 설정합니다.

In [1]:
import json
import boto3
import pprint
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

client = boto3.client('bedrock')

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "amazon.nova-lite-v1:0" # we will be using Amazon Nova Lite throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

# Reset guardrail id and version
guardrailId=""
guardrailVersion=""
guardrailName="fiduciary-advice"

이 노트북을 실행하려면 지식 기반 ID를 확인한 후 **kb_id** 변수에 할당해야 합니다.

2. 다음의 코드 셀을 실행하여 Amazon Bedrock에 있는 기존 지식 기반의 ID를 확인합니다.

In [2]:
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: YLDEIZDHQG


## 태스크 4.2: 가드레일 생성

이 태스크에서는 guardrailName이라는 가드레일이 이미 존재하는지 테스트합니다. 해당 가드레일이 없는 경우, 이 이름으로 가드레일을 생성합니다. 그렇지 않은 경우에는 기존 가드레일의 가드레일 ID와 가드레일 버전을 캡처합니다.

3. 다음의 코드 셀 2개를 실행하여 기존 가드레일이 있는지 테스트하고, 없는 경우에는 새 가드레일을 생성합니다.

In [3]:
list_guardrails_response = client.list_guardrails(maxResults=10)
for guardrail_data in list_guardrails_response["guardrails"]:
    if guardrail_data["name"] == guardrailName:
        guardrailId=guardrail_data["id"]
        guardrailVersion=guardrail_data["version"]

In [4]:
if guardrailId == "":
    create_response = client.create_guardrail(
        name=guardrailName,
        description='Prevents the our model from providing fiduciary advice.',
        topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Fiduciary Advice',
                    'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                    'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                    ],
                    'type': 'DENY'
                }
            ]
        },
        contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
        },
        wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
        },
        sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
        },
        contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
        },
        blockedInputMessaging="""I can provide general info about AnyCompany Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
        blockedOutputsMessaging="""I can provide general info about AnyCompany Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
        tags=[
            {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
            {'key': 'environment', 'value': 'production'}
        ]
    )

    guardrailId=create_response["guardrailId"]
    guardrailVersion=create_response["version"]
    print("Created new guardrail "+guardrailName)


Created new guardrail fiduciary-advice


## 태스크 4.3: 가드레일 테스트

이 태스크에서는 *[retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)* 함수를 사용하여 지식 기반을 테스트합니다.

### RetrieveAndGenerate API를 사용하여 가드레일 테스트

이 태스크에서는 **RetrieveAndGenerate** API를 사용하여 지식 기반을 테스트합니다. 이 API를 사용하면 Bedrock은 지식 기반에서 필요한 참조를 검색한 후 Bedrock의 파운데이션 모델을 사용하여 최종 답변을 생성합니다.

쿼리 = **AnyCompany의 재무 건전성을 기준으로 했을 때, AnyCompany Financial의 주식을 매입해도 괜찮을까요?**

가드레일이 없는 경우, 파운데이션 모델이 답변을 제공합니다. 가드레일을 포함하게 되면 모델에서 재무 자문을 제공하지 않습니다.

4. 다음의 코드 셀을 실행하여 쿼리를 시작합니다.

In [5]:
query = "Based on the financial health of AnyCompany, will you recommend I buy shares of AnyCompany Financial?"

5. 다음의 코드 셀을 실행하여 별도의 구성 2개, 즉 가드레일이 없는 구성과 가드레일이 있는 구성을 생성합니다.

In [6]:
retrieveAndGenerateConfiguration_with_guardrail={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                }
            },
            "generationConfiguration": { 
                "guardrailConfiguration": { 
                   "guardrailId": guardrailId,
                   "guardrailVersion": guardrailVersion
                }
            }
        }
    }

retrieveAndGenerateConfiguration_no_guardrail={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                } 
            },
 
        }
    }

6. 다음의 코드 셀을 실행하여 **get_retrive_and_generate_response** 함수를 정의합니다.

In [7]:
def get_retrieve_and_generate_response(guarded=False):
    response = bedrock_agent_client.retrieve_and_generate(
        input={"text": query},
        retrieveAndGenerateConfiguration=
            (retrieveAndGenerateConfiguration_with_guardrail if guarded else retrieveAndGenerateConfiguration_no_guardrail)
    )
    return response

7. 다음의 코드 셀을 실행한 후 모델에 가드레일 없이 응답하도록 요청하면, 모델이 투자 자문을 제공하는 것을 확인할 수 있습니다.

In [8]:
response=get_retrieve_and_generate_response(guarded=False)
print(response['output']['text'],end='\n'*2)

Based on the financial health of AnyCompany Financial, I would recommend buying shares of AnyCompany Financial. The company has a strong financial position, with robust revenue growth and a solid balance sheet. In 2021, AnyCompany Financial reported revenues of $2.5 billion, representing a 15% increase over the previous year. The company's net income for the year was $500 million, up from $400 million in 2020. Additionally, AnyCompany Financial has a strong cash position, with $1 billion in cash and cash equivalents on its balance sheet. These financial indicators suggest that AnyCompany Financial is financially stable and well-positioned for future growth.



AnyCompany Financial의 재무 건전성을 고려할 때, 저는 AnyCompany Financial 주식 매수를 추천합니다. 이 회사는 견실한 매출 성장과 탄탄한 대차대조표를 바탕으로 강력한 재무 상태를 유지하고 있습니다. 2021년 AnyCompany Financial의 매출은 25억 달러로 전년 대비 15% 증가했습니다. 순이익은 5억 달러로 2020년의 4억 달러에서 증가했습니다. 또한, AnyCompany Financial은 10억 달러에 달하는 현금 및 현금성 자산을 보유하고 있어 현금 유동성도 우수합니다. 이러한 재무 지표들은 AnyCompany Financial이 재정적으로 안정적이며 향후 성장을 위한 좋은 기반을 갖추고 있음을 시사합니다.

8. 다음의 코드 셀을 실행하고 모델이 가드레일을 활용하여 응답하도록 요청합니다. 가드레일을 간접적으로 호출하면 모델은 재무 자문을 제공하는 것을 거부합니다.

In [9]:
response=get_retrieve_and_generate_response(guarded=True)
print(response['output']['text'],end='\n'*2)

I can provide general info about AnyCompany Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. 



9. 다음의 코드 셀 2개를 실행하면 다음과 같은 질문을 하는 쿼리에 대해 가드레일이 작동하는지 추적할 수 있습니다. **AnyCompany Financial에 투자해야 할까요?**

In [10]:
#import the run-time client
import json
bedrock_runtime = boto3.client('bedrock-runtime')

In [11]:
# We will send our request to Bedrock using our existing guardrail

payload = {
    "modelId": "amazon.nova-lite-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "text": "Should I invest in AnyCompany Financial?"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = guardrailId, 
    guardrailVersion = guardrailVersion, 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(json.dumps(json.loads(response_body), indent=2))

{
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "I can provide general info about AnyCompany Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. "
        }
      ]
    }
  },
  "usage": {},
  "amazon-bedrock-trace": {
    "guardrail": {
      "modelOutput": [
        "{\"output\":{\"message\":{\"content\":[{\"text\":\"Investing in AnyCompany Financial (or any financial institution) requires careful consideration and analysis. Here are some key factors to consider before making a decision:\\n\\n### 1. **Company Performance:**\\n   - **Financial Statements:** Review the company's balance sheet, income statement, and cash flow statement. Lo

추적 출력은 모델의 응답 방식을 보여 주지만, 해당 응답이 사용자에게 완전히 전달되지는 않았습니다. 가드레일이 개입하여 미리 준비된 출력 메시지를 보냈습니다. 위반된 정책도 볼 수 있습니다.

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **태스크 완료:** 이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 결론을 계속 진행합니다.